<a href="https://colab.research.google.com/github/ashikita/openalex-api-notebook/blob/main/cited_by_count_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pyalexを使ってOpenAlexから論文情報を取得します。

In [27]:
import pyalex
from pyalex import Works
import csv

# === 設定項目 ===
ROR_ID = "00p4k0j84"  # 九州大学のROR番号
FROM_DATE = "2024-01-01"
TO_DATE = "2024-12-31"
MAX_RESULTS = 10
OUTPUT_FILE = "output.tsv"

# OpenAlex API設定
pyalex.config.email = "hogehoge@m.kyushu-u.ac.jp"

# データ取得
results = Works() \
    .filter(
        authorships={"institutions": {"ror": ROR_ID}},
        from_publication_date=FROM_DATE,
        to_publication_date=TO_DATE
    ) \
    .sort(cited_by_count="desc") \
    .get(per_page=MAX_RESULTS)

# 必要な要素を抽出
filtered_results = []
for work in results:
    corresponding_authors = []
    for authorship in work.get("authorships", []):
        if authorship.get("is_corresponding"):
            for institution in authorship.get("institutions", []):
                if institution.get("ror") == f"https://ror.org/{ROR_ID}":
                    display_name = authorship.get("author", {}).get("display_name")
                    if display_name:
                        corresponding_authors.append(display_name)
                    break

    filtered_results.append({
        "doi": work.get("doi"),
        "publication_date": work.get("publication_date"),
        "cited_by_count": work.get("cited_by_count"),
        "is_oa": work.get("open_access", {}).get("is_oa"),
        "oa_status": work.get("open_access", {}).get("oa_status"),
        "oa_url": work.get("open_access", {}).get("oa_url"),
        "any_repository_has_fulltext": work.get("open_access", {}).get("any_repository_has_fulltext"),
        "corresponding_authors": ", ".join(corresponding_authors)
    })

# TSVファイルに保存
with open(OUTPUT_FILE, "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "doi", "publication_date", "cited_by_count",
        "is_oa", "oa_status", "oa_url", "any_repository_has_fulltext",
        "corresponding_authors"
    ], delimiter='\t')
    writer.writeheader()
    writer.writerows(filtered_results)

print(f"✅ データは {OUTPUT_FILE} に保存されました。")

✅ データは output.tsv に保存されました。
